# Brevíssima introdução à utilização do Z3 em Python

Vamos usar como SMT solver o popular solver Z3, através da biblioteca Python Z3Py.
A documentação do Z3py pode ser encontrada em https://ericpony.github.io/z3py-tutorial/guide-examples.htm.


Começamos por instalar o Z3.

In [1]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 11.1 MB/s eta 0:00:00


Em https://theory.stanford.edu/~nikolaj/programmingz3.html tem um tutorial bastante completo do Z3.

A utilização da biblioteca Z3py segue normalmente os seguintes passos:

1. Importar a biblioteca do Z3 usando o comando `from z3 import *`
1. Criar uma instância do solver com a função `Solver()`.
1. Adicionar as variáveis do problema. As funções `Int()`, `Real()`, `Bool()` criam uma variável no Z3 do tipo correspondente. Estas funções recebem o nome da variável como parâmetro.
1. Adicionar as restrições do problema usando o método `add`. A restrição é definida com a sintaxe normal Python para expressões aritméticas e comparações.  As funções `And`, `Or`, `Not` e `Implies` são usadas como os operadores lógicos.

1. Invocar o solver com o método `check`. Este método pode devolver um dos seguintes resultados:
  - `sat`, quando o conjunto de restrições é satisfazível, isto é, quando uma solução for encontrada.
  - `unsat`, quando não é possível resolver o problema, isto é, quando o conjunto de restrições é inconsistente.
  - `unknown`, quando o solver não se consegue pronunciar quanto satisfatibilidade do conjunto de restrições.

1. Interpretar os resultados no caso do resultado ser `sat`. O método `model` dá o modelo produzido pelo solver (o modelo associa a cada variável o valor que lhe foi atribuido na solução encontrada).
Podemos consultar o modelo gerado. O método `decls` devolve as variáveis atribuídas no modelo. O método `name` devolve o nome de uma variável atribuída no modelo. O método `eval` permite avaliar uma expressão no modelo.
Deve ser usado o método `as_long` para converter os valores inteiros do Z3 em inteiros do Python.



Por exemplo, o programa que se segue tenta encontrar solução para as seguintes restrições:

$$
\left\{
\begin{array}{l}
0 \le x \le 50\\
y \ge x+z \\
z-5y > x \vee 3x+y \le 20
\end{array}
\right.
$$



In [2]:
from z3 import *

s = Solver()

x, y = Ints('x y')
z = Int('z')

s.add(And(0<=x,x<=50))
s.add(y >= x+z)
s.add(Or(z-5*y>x, 3*x+y<=20))

if s.check()==sat :
    m = s.model()
    print(m)
    for d in m.decls():
        print("%s = %d" % (d.name(), m[d].as_long()))
else:
    print("Não tem solução.")

[x = 50, z = -175, y = -125]
x = 50
z = -175
y = -125


Em algumas aplicações, queremos explorar vários problemas semelhantes que compartilham várias restrições. Podemos usar os métodos `push` e `pop` para fazer isso. Cada solver mantém uma pilha de asserções (restrições). O método `push` cria um novo escopo, salvando o tamanho atual da pilha. O método  `pop` remove qualquer asserção acrescentada entre ele e o `push` correspondente. O método `check` opera sobre o conjunto de asserções que está no topo da pilha.

In [3]:
print(s)
print("Solving constraints in the solver s ...")
print(s.check())

[And(x >= 0, x <= 50),
 y >= x + z,
 Or(z - 5*y > x, 3*x + y <= 20)]
Solving constraints in the solver s ...
sat


In [4]:
print("Create a new scope...")
s.push()
s.add( x > 110)
print(s)
print("Solving updated set of constraints...")
print(s.check())

Create a new scope...
[And(x >= 0, x <= 50),
 y >= x + z,
 Or(z - 5*y > x, 3*x + y <= 20),
 x > 110]
Solving updated set of constraints...
unsat


In [5]:
print("Restoring state...")
s.pop()
print(s)
print("Solving restored set of constraints...")
print(s.check())

Restoring state...
[And(x >= 0, x <= 50),
 y >= x + z,
 Or(z - 5*y > x, 3*x + y <= 20)]
Solving restored set of constraints...
sat


Para já vamos apenas usar o Z3 como SAT solver, pegando em dois exemplos de modelação em lógica proposicional já trabalhados na aula.

### Placement of guests

Considere o seguinte problema.



> *Temos 3 cadeiras em linha (esquerda, meio, e direita) e precisamos de sentar 3 convidados: a Ana, a Susana e o Pedro. No entanto:*
- *A Ana não quer ficar sentada à beira do Pedro.*
- *A Ana não quer ficar na cadeira da esquerda.*
- *A Susana não se quer sentar à esquerda do Pedro.*
>
> *Será possível sentar os convidados? Como?*


Modele o problema em Lógica Proposicional e use o Z3 para o resolver. Não se esqueça que todas as pessoas devem ficar sentadas e que só é possível sentar uma pessoa por cadeira.

**Sugestão:** Crie uma variável proposicional (com nome sugestivo) para cada par $(p,c)$, onde $p$ é uma pessoa e $c$ uma cadeira. Se a pessoa $p$ ficar sentada na cadeira $c$ o valor da variável respectiva será `True`, caso contrário será `False`.

In [8]:
pessoas = ["Ana","Susana","Pedro"]
cadeiras = ["esquerda","meio","direita"]
v = {}
for p in pessoas:
    v[p] = {}
    for c in cadeiras:
        v[p][c] = Bool("%s,%s" % (p,c))

s = Solver()

# A Ana não quer ficar sentada à beira do Pedro.
s.add(Implies(Or(v["Ana"]["esquerda"],v["Ana"]["direita"]),Not(v["Pedro"]["meio"])))
s.add(Implies(v["Ana"]["meio"],And(Not(v["Pedro"]["esquerda"]),Not(v["Pedro"]["direita"]))))

# A Ana não quer ficar na cadeira da esquerda.
s.add(Not(v["Ana"]["esquerda"]))

# A Susana não se quer sentar à esquerda do Pedro.
s.add(Implies(v["Pedro"]["direita"],Not(v["Susana"]["meio"])))
s.add(Implies(v["Pedro"]["meio"],Not(v["Susana"]["esquerda"])))

# Cada pessoa está sentada.
for p in pessoas:
   s.add(Or([v[p][c] for c in cadeiras]))

# Apenas uma pessoa por cadeira.
for c in cadeiras:
    for i in range(len(pessoas)-1):
        for j in range(i+1,len(pessoas)):
            s.add(Implies(v[pessoas[i]][c],Not(v[pessoas[j]][c])))

if s.check() == sat:
    m = s.model()
    #print(m)
    for p in pessoas:
        for c in cadeiras:
            if is_true(m[v[p][c]]):
                print("%s senta-se na cadeira %s" % (p,c))

Ana senta-se na cadeira direita
Susana senta-se na cadeira meio
Pedro senta-se na cadeira esquerda


### Unicorn puzzle

Recorde o enigma do unicórnio:

> - *If the unicorn is mythical, then it is immortal.*
- *If the unicorn is not mythical, then it is a mortal mammal.*
- *If the unicorn is either immortal or a mammal, then it is horned.*
- *The unicorn is magical if it is horned.*
>
> *Given these constraints:*
- *Is the unicorn magical? *
- *Is it horned?*
- *Is it mythical?*
- *Is it possible for the unicorn to be simultaneously mythical and
immortal?*


Modele o problema em Lógica Proposicional e use o Z3 para o resolver.

**Sugestão:** Resolva o problema com o auxílio de 5 variáveis proposicionais, correspondentes às 5 propriedades dos unicórnios. Relembre que a afirmação $A_1, \ldots, A_n \models B$ é válida se e só se o conjunto de restrições $\{A_1, \ldots, A_n, \neg B\}$ é inconsistente. Tire proveito dos métodos `push` e `pop` para responder às várias questões usando de forma incremental o mesmo solver.

In [9]:
mythical, immortal, mammal = Bools('mythical immortal mammal')
horned, magical = Bools('horned magical')

s = Solver()

# If the unicorn is mythical, then it is immortal.
s.add(Implies(mythical,immortal))

# If the unicorn is not mythical, then it is a mortal mammal.
s.add(Implies(Not(mythical),And(Not(immortal),mammal)))

# If the unicorn is either immortal or a mammal, then it is horned.
s.add(Implies(Or(immortal,mammal),horned))

# The unicorn is magical if it is horned.
s.add(Implies(horned,magical))

s.push()

print("Is the unicorn magical?")
s.add(Not(magical))

r = s.check()
if r == unsat:
    print('Yes.')
else:
    print('Not necessarily.')

s.pop()
s.push()

print("\nIs the unicorn horned?")
s.add(Not(horned))

r = s.check()
if r == unsat:
    print('Yes.')
else:
    print('Not necessarily.')

s.pop()
s.push()


print("\nIs the unicorn mythical?")
s.add(Not(mythical))

r = s.check()
if r == unsat:
    print('Yes.')
else:
    print('Not necessarily.')
    print("The unicorn can be not mythical...")

s.pop()
s.push()

s.add(mythical)

r = s.check()
if r == unsat:
    print('More, he is mythical for sure.')
else:
    print("but can also be mythical.")

s.pop()
s.push()

print("\nIs it possible for the unicorn to be simultaneously mythical and immortal?")
s.add(And(mythical, immortal))

if s.check() == sat:
    print('Yes, it is possible.')
else:
    print('No, it is not possible.')


Is the unicorn magical?
Yes.

Is the unicorn horned?
Yes.

Is the unicorn mythical?
Not necessarily.
The unicorn can be not mythical...
but can also be mythical.

Is it possible for the unicorn to be simultaneously mythical and immortal?
Yes, it is possible.
